In [1]:
from utils import getCorpus
from modeling import Models
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from input_fn import Dataset
import pandas as pd

In [2]:
positivecorpus = getCorpus('https://storm.cis.fordham.edu/~yli/data/electronics/positive.review','review_text')
negativecorpus = getCorpus('https://storm.cis.fordham.edu/~yli/data/electronics/negative.review','review_text')
ps = PorterStemmer() 
vectorizer = CountVectorizer()

In [3]:
p = Dataset(corpus=positivecorpus, tokenizer=word_tokenize, stemmer=ps,keep_pantuation= False)
n = Dataset(corpus=negativecorpus, tokenizer=word_tokenize, stemmer=ps,keep_pantuation= False)

In [4]:
p.getmostwords(20)

[('works', 184),
 ('ipod', 185),
 ('ive', 193),
 ('used', 202),
 ('phone', 203),
 ('bought', 206),
 ('price', 217),
 ('dont', 224),
 ('product', 230),
 ('well', 236),
 ('also', 240),
 ('get', 256),
 ('would', 258),
 ('quality', 274),
 ('like', 284),
 ('sound', 355),
 ('good', 399),
 ('great', 402),
 ('one', 429),
 ('use', 434)]

In [5]:
# Get Matrix
df_positive = p.getdata(p.X,y = 1)
df_negative = n.getdata(p.X,y = 0)
df = pd.concat([df_positive,df_negative],axis=0)
df = df.sample(frac=1).reset_index(drop=True)
df['review'] = df['review'].apply(lambda x : " ".join(x))

In [6]:
## Modeling
x_train,x_test,y_train,y_test = train_test_split(df.review,df.sentiment,test_size = 0.3)

v_x_train = vectorizer.fit_transform(x_train)
v_x_test = vectorizer.transform(x_test)
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
models = Models(v_x_train,y_train,kfolds)
log,nb = models.main()

START Fit
2019-09-28 23:23:39.418045 NB
Training done !!
2019-09-28 23:23:39.422339 logistics
Training done !!
